# We have run convergence studies and found a few different modes 

Now we see about their orthogonality properties

In [ ]:
import ngsolve as ng
import numpy as np

from fiberamp.fiber.microstruct.pbg import ARF2
from ngsolve.webgui import Draw

# Load the modes

In [ ]:
ref = 0
p = 14
a = ARF2(refine=ref, curve=max(p+1,3), poly_core=True)


In [ ]:
# HE11
Es_path = 'HE11/modes/Es_ref' + str(ref) + 'p' + str(p)
phis_path = 'HE11/modes/phis_ref' + str(ref) + 'p' + str(p)
betas_path = 'HE11/outputs/ref' + str(ref) + 'p' + str(p) + 'betas.npy'

HE11_tv = a.load_E_modes(mesh_path, Es_path, p=p)
HE11_phi = a.load_phi_modes(mesh_path, phis_path, p=p)
HE11_betas = np.load(betas_path)

In [ ]:
# TE
Es_path = 'TE/modes/Es_ref' + str(ref) + 'p' + str(p)
phis_path = 'TE/modes/phis_ref' + str(ref) + 'p' + str(p)
betas_path = 'TE/outputs/ref' + str(ref) + 'p' + str(p) + 'betas.npy'

TE_tv = a.load_E_modes(mesh_path, Es_path, p=p)
TE_phi = a.load_phi_modes(mesh_path, phis_path, p=p)
TE_betas = np.load(betas_path)

In [ ]:
# HOMs
Es_path = 'HOMS/modes/Es_ref' + str(ref) + 'p' + str(p)
phis_path = 'HOMS/modes/phis_ref' + str(ref) + 'p' + str(p)
betas_path = 'HOMS/outputs/ref' + str(ref) + 'p' + str(p) + 'betas.npy'

HOMS_tv = a.load_E_modes(mesh_path, Es_path, p=p)
HOMS_phi = a.load_phi_modes(mesh_path, phis_path, p=p)
HOMS_betas = np.load(betas_path)

# Draw Modes

## HE11

In [ ]:
for e in HE11_tv:
    Draw(e.real, a.mesh, vectors={'grid_size':300})

## TE

In [ ]:
for e in TE_tv:
    Draw(e.real, a.mesh, vectors={'grid_size':300})

## HOMS

In [ ]:
for e in HOMS_tv:
    Draw(e.real, a.mesh, vectors={'grid_size':300})

# Orthogonality Check



In [ ]:
# Set choice for fundamental mode
Etv_fund, phi_fund, beta_fund = HE11_tv[0], HE11_phi[0], HE11_betas[0]

# Get H field for fundamental mode
Htv_fund, Hz_fund = A.HfromE(Etv_fund, phi_fund, beta_fund)

# Find current modal power
E_fund_power = A.power(Etv_fund, Htv_fund)
E_fund_power

In [ ]:
# Set choice for higher order mode (here Transverse Electric type)
Etv_TE, Ephi_TE, beta_TE = TE_tv[2], TE_phi[2], TE_betas[2]

# Get H field for hom
Htv_TE, Hz_TE = A.HfromE(Etv_TE, Ephi_TE, beta_TE)

# Find current modal power
E_TE_power = A.power(Etv_TE, Htv_TE)

E_TE_power

In [ ]:
# Set choice for higher order mode (here Transverse Electric type)
Etv_HOM, Ephi_HOM, beta_HOM = HOMS_tv[0], HOMS_phi[0], HOMS_betas[2]

# Get H field for hom
Htv_HOM, Hz_HOM = A.HfromE(Etv_HOM, Ephi_HOM, beta_HOM)

# Find current modal power
E_hom_power = A.power(Etv_HOM, Htv_HOM)

E_hom_power

In [ ]:
# Un-normalized inner product
fund_TE_ip = A.power(Etv_fund, Htv_TE)

In [ ]:
# Normalized inner product
fund_TE_ip/ (E_fund_power * E_TE_power)**.5

In [ ]:
# Un-normalized inner product
fund_HOM_ip = A.power(Etv_fund, Htv_HOM)

In [ ]:
# Normalized inner product
fund_HOM_ip/ (E_fund_power * E_hom_power)**.5

In [ ]:
# Un-normalized inner productL
HOM_TE_ip = A.power(Etv_HOM, Htv_TE)

In [ ]:
# Normalized inner product
HOM_TE_ip/ (E_hom_power * E_TE_power)**.5

# Other types of orthogonality

In [ ]:
# Create full E field for fundamental
Ez_fund = -1j/beta_fund * phi_fund
E_fund_full = ng.CoefficientFunction((Etv_fund[0], Etv_fund[1], Ez_fund))

# Create full H field for fundamental
H_fund_full = ng.CoefficientFunction((Htv_fund[0], Htv_fund[1], Hz_fund))


# Create full E field for TE mode
Ez_TE = -1j/beta_TE * Ephi_TE
E_TE_full = ng.CoefficientFunction((Etv_TE[0], Etv_TE[1], Ez_TE))

# Create full hom H field
H_TE_full = ng.CoefficientFunction((Htv_TE[0], Htv_TE[1], Hz_TE))


# Create full E field for higher order mode
Ez_HOM = -1j/beta_HOM * Ephi_HOM
E_HOM_full = ng.CoefficientFunction((Etv_HOM[0], Etv_HOM[1], Ez_HOM))

# Create full hom H field
H_HOM_full = ng.CoefficientFunction((Htv_HOM[0], Htv_HOM[1], Hz_HOM))



# Full Field dot product Orthogonality

In [ ]:
HE11_norm_sqrd = ng.Integrate(ng.Norm(E_fund_full)**2, A.mesh) + ng.Integrate(ng.Norm(H_fund_full)**2, A.mesh)


In [ ]:
TE_norm_sqrd = ng.Integrate(ng.Norm(E_TE_full)**2, A.mesh) + ng.Integrate(ng.Norm(H_TE_full)**2, A.mesh)


In [ ]:
HOM_norm_sqrd = ng.Integrate(ng.Norm(E_HOM_full)**2, A.mesh) + ng.Integrate(ng.Norm(H_HOM_full)**2, A.mesh)


In [ ]:
t1 = ng.InnerProduct(E_fund_full, E_TE_full)
t2 = ng.InnerProduct(H_fund_full, H_TE_full)
HE11_TE_full = ng.Integrate(t1 + t2, A.mesh)

In [ ]:
HE11_TE_full / (HE11_norm_sqrd * TE_norm_sqrd)**.5

In [ ]:
t1 = ng.InnerProduct(E_fund_full, E_HOM_full)
t2 = ng.InnerProduct(H_fund_full, H_HOM_full)
HE11_HOM_full = ng.Integrate(t1 + t2, A.mesh)

In [ ]:
HE11_HOM_full / (HE11_norm_sqrd * HOM_norm_sqrd)**.5

# Component-wise Orthogonality

In [ ]:
# E field componentwise 

for i in range(3):
    print(ng.Integrate(ng.InnerProduct(E_fund_full[i], E_TE_full[i]), A.mesh))

In [ ]:
# H field componentwise 

for i in range(3):
    print(ng.Integrate(ng.InnerProduct(H_fund_full[i], H_TE_full[i]), A.mesh))

In [ ]:
# Mixed

for i in range(3):
    print(ng.Integrate(ng.InnerProduct(E_fund_full[i], H_TE_full[i]), A.mesh))